In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [5]:
X_train = pd.read_csv("../data/madelon_train.data",delimiter=" ",header=None).iloc[:,:-1].values
y_train = pd.read_csv("../data/madelon_train.labels",delimiter=" ",header=None).values.flatten()
y_train[y_train==-1] = 0

X_test = pd.read_csv("../data/madelon_valid.data",delimiter=" ",header=None).iloc[:,:-1].values
y_test = pd.read_csv("../data/madelon_valid.labels",delimiter=" ",header=None).values.flatten()
y_test[y_test==-1] = 0

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
X = np.concatenate([X_train,X_test])
y = np.concatenate([y_train,y_test])
train_size = 2000

dataset = 'madelon'

In [6]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

def initial(y_one_hot):
    init_log_odds = np.sum(y_one_hot,axis=0)/y_one_hot.shape[0]
    
    init_p = np.exp(init_log_odds)
    init_p = init_p/(1+init_p)

    return init_log_odds,init_p

y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

# model

In [11]:
def retain(gtgp):
    retrain_epoch=1000
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators_roc(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

retain(gtgp)

retrain  1 :
	train: 0.7251236943375481 0.8026958739830026 	test: 0.70550576184379 0.7764443570070167
retrain  2 :
	train: 0.7520615722924684 0.8292085252481292 	test: 0.7221510883482715 0.8009902288674668
retrain  3 :
	train: 0.7663551401869159 0.8476142119706476 	test: 0.737516005121639 0.8168994688176274
retrain  4 :
	train: 0.777350192413414 0.8619192688499617 	test: 0.7503201024327785 0.8293068397927733
retrain  5 :
	train: 0.7905442550852116 0.8730823631813731 	test: 0.7592829705505761 0.8387894288150043
retrain  6 :
	train: 0.8064870808136338 0.882098429623182 	test: 0.7746478873239436 0.8460226900124597
retrain  7 :
	train: 0.8152831225948324 0.8896517124239898 	test: 0.7810499359795134 0.8526067283100531
retrain  8 :
	train: 0.817482133040132 0.8957289135506958 	test: 0.7810499359795134 0.8580431503705162
retrain  9 :
	train: 0.8218801539307312 0.901071096120601 	test: 0.7874519846350833 0.8623516296150567
retrain  10 :
	train: 0.8257284222100055 0.9053421825699053 	test: 0.78

KeyboardInterrupt: 

In [15]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=1

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=30
    elite_size = 10
    epoch= 1000
    gp_epoch= 30
    verbose = 1
    tolerance=0.001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=200
    alpha=1
    beta=1
    gammer=1

    verbose=1
    gtgp.lam = 1000
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_roc = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    test_roc = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)


    # with open('./benchmark/'+dataset+'.csv','a') as f:
    #     s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
    #     f.writelines(s)

92 432 1
131 599 2
172 776 3
206 902 4
231 949 5
249 981 6
265 1035 7
290 1160 8
300 1180 9
310 1192 10
322 1248 11
336 1304 12
347 1325 13
359 1385 14
364 1414 15
379 1487 16
381 1467 17
386 1484 18
392 1494 19
398 1524 20
403 1521 21
410 1544 22
419 1571 23
428 1610 24
432 1606 25
435 1611 26
440 1624 27
445 1641 28
447 1657 29
453 1679 30
456 1674 31
459 1663 32
463 1667 33
475 1737 34
479 1751 35
480 1754 36
482 1756 37
485 1765 38
487 1753 39
497 1817 40
497 1811 41
498 1810 42
499 1801 43
502 1816 44
505 1819 45
508 1828 46
513 1845 47
515 1847 48
522 1866 49
525 1875 50
527 1871 51
530 1890 52
531 1891 53
532 1896 54
538 1940 55
539 1943 56
540 1956 57
541 1935 58
542 1942 59
544 1950 60
546 1948 61
553 1989 62
554 1990 63
554 1990 64
555 2001 65
557 2007 66
561 2009 67
560 1982 68
571 2051 69
572 2048 70
574 2044 71
575 2043 72
577 2043 73
577 2029 74
580 2044 75
584 2056 76
584 2048 77
588 2068 78
589 2075 79
592 2090 80
596 2106 81
598 2112 82
600 2136 83
601 2137 84
602 2134

KeyboardInterrupt: 

In [ ]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=1

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=300
    elite_size = 30
    epoch= 100
    gp_epoch= 15
    verbose = 1
    tolerance=0.001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=5
    alpha=1
    beta=1
    gammer=1

    verbose=1
    gtgp.lam = 100
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(30):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_roc = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    test_roc = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

153 1247 1
253 2175 2
325 2869 3
383 3359 4
419 3675 5
464 4056 6
477 4049 7
497 4125 8
553 4795 9
570 4800 10
600 5044 11
623 5197 12
644 5338 13
645 5255 14
657 5445 15
667 5405 16
675 5413 17
695 5559 18
706 5634 19
716 5652 20
730 5726 21
736 5658 22
742 5616 23
748 5644 24
757 5797 25
786 5914 26
792 5904 27
805 5987 28
813 5897 29
820 5940 30
819 5917 31
824 5962 32
837 6041 33
837 5987 34
843 6091 35
849 6099 36
854 6102 37
865 6017 38
871 6105 39
879 6297 40
890 6542 41
891 6533 42
891 6503 43
892 6454 44
899 6477 45
902 6458 46
907 6481 47
908 6518 48
916 6578 49
919 6603 50
919 6569 51
924 6582 52
933 6623 53
939 6663 54
954 6992 55
954 6926 56
954 6870 57
957 6871 58
957 6855 59
962 6906 60
965 6949 61
961 6881 62
965 6855 63
972 6938 64
980 6948 65
983 6955 66
986 6942 67
999 7127 68
1002 7130 69
1000 6998 70
1001 6995 71
1011 7047 72
1019 7183 73
1023 7211 74
1027 7233 75
1028 7228 76
1027 7157 77
1026 7142 78
1028 7148 79
1036 7194 80
1036 7162 81
1042 7190 82
1042 7190 8

In [55]:
retrain_epoch=100
alpha=0
beta=0
gammer=0

verbose=1
gtgp.lam = 100

gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

retrain  1 :
	train: 0.6385 885.8614308975286 	test: 0.6216666666666667 272.50651497731826
retrain  2 :
	train: 0.658 845.7862548638049 	test: 0.6366666666666667 263.523460037829
retrain  3 :
	train: 0.677 818.5219343389831 	test: 0.665 257.35617116744044
retrain  4 :
	train: 0.696 796.0633899665838 	test: 0.6783333333333333 252.30869866510886
retrain  5 :
	train: 0.711 776.9651940184807 	test: 0.6916666666666667 248.1197740521675
retrain  6 :
	train: 0.7205 760.5360046357259 	test: 0.7016666666666667 244.6362363167052
retrain  7 :
	train: 0.729 746.2694534022057 	test: 0.7066666666666667 241.72925834531964
retrain  8 :
	train: 0.7355 733.7712404102268 	test: 0.715 239.29326375073992
retrain  9 :
	train: 0.7425 722.7329217491285 	test: 0.715 237.2438968376748
retrain  10 :
	train: 0.7485 712.9120890963663 	test: 0.7166666666666667 235.5141297344499
retrain  11 :
	train: 0.7515 704.1165839673329 	test: 0.7216666666666667 234.0504741843058
retrain  12 :
	train: 0.756 696.1924561754865 	t

KeyboardInterrupt: 

In [121]:
gtgp.print_model()
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))
p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

----------------GTGP-------------
Number of Trees: 1318
Average of depth: 1.24051593323217
Number of nodes: 4596
              precision    recall  f1-score   support

           0    0.99900   1.00000   0.99950      1000
           1    1.00000   0.99900   0.99950      1000

    accuracy                        0.99950      2000
   macro avg    0.99950   0.99950   0.99950      2000
weighted avg    0.99950   0.99950   0.99950      2000

              precision    recall  f1-score   support

           0    0.72575   0.72333   0.72454       300
           1    0.72425   0.72667   0.72546       300

    accuracy                        0.72500       600
   macro avg    0.72500   0.72500   0.72500       600
weighted avg    0.72500   0.72500   0.72500       600



(1.0, 0.8015888888888889)

In [77]:
gtgp.print_model()
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))
p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

----------------GTGP-------------
Number of Trees: 519
Average of depth: 2.984585741811175
Number of nodes: 5363
              precision    recall  f1-score   support

           0    0.98703   0.98900   0.98801      1000
           1    0.98898   0.98700   0.98799      1000

    accuracy                        0.98800      2000
   macro avg    0.98800   0.98800   0.98800      2000
weighted avg    0.98800   0.98800   0.98800      2000

              precision    recall  f1-score   support

           0    0.83607   0.85000   0.84298       300
           1    0.84746   0.83333   0.84034       300

    accuracy                        0.84167       600
   macro avg    0.84176   0.84167   0.84166       600
weighted avg    0.84176   0.84167   0.84166       600



(0.999061, 0.920188888888889)

In [14]:
gtgp.print_model()
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))
p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

----------------GTGP-------------
Number of Trees: 664
Average of depth: 1.1024096385542168
Number of nodes: 2146
              precision    recall  f1-score   support

           0    0.97114   0.97600   0.97357      1000
           1    0.97588   0.97100   0.97343      1000

    accuracy                        0.97350      2000
   macro avg    0.97351   0.97350   0.97350      2000
weighted avg    0.97351   0.97350   0.97350      2000

              precision    recall  f1-score   support

           0    0.78912   0.77333   0.78114       300
           1    0.77778   0.79333   0.78548       300

    accuracy                        0.78333       600
   macro avg    0.78345   0.78333   0.78331       600
weighted avg    0.78345   0.78333   0.78331       600



(0.9965809999999999, 0.8515111111111111)

In [408]:
gtgp.print_model()

----------------GTGP-------------
Number of Trees: 24
Number of nodes: 94


In [409]:
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

              precision    recall  f1-score   support

           0    0.78926   0.79400   0.79163      1000
           1    0.79276   0.78800   0.79037      1000

    accuracy                        0.79100      2000
   macro avg    0.79101   0.79100   0.79100      2000
weighted avg    0.79101   0.79100   0.79100      2000

              precision    recall  f1-score   support

           0    0.77320   0.75000   0.76142       300
           1    0.75728   0.78000   0.76847       300

    accuracy                        0.76500       600
   macro avg    0.76524   0.76500   0.76495       600
weighted avg    0.76524   0.76500   0.76495       600



In [51]:
gtgp.print_model()

----------------GTGP-------------
Number of Trees: 17
Number of nodes: 57


In [52]:
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

              precision    recall  f1-score   support

           0    0.72634   0.77500   0.74988      1000
           1    0.75884   0.70800   0.73254      1000

    accuracy                        0.74150      2000
   macro avg    0.74259   0.74150   0.74121      2000
weighted avg    0.74259   0.74150   0.74121      2000

              precision    recall  f1-score   support

           0    0.73734   0.77667   0.75649       300
           1    0.76408   0.72333   0.74315       300

    accuracy                        0.75000       600
   macro avg    0.75071   0.75000   0.74982       600
weighted avg    0.75071   0.75000   0.74982       600



# Decision Tree

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    # y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

NameError: name 'train_test_split' is not defined

# XGBoost

In [13]:
import xgboost


In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=3,n_estimators=100)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
    xgb = xgboost.XGBClassifier(n_estimators=1000)
    xgb.fit(X_train,y_train)

    import json

    def item_generator(json_input, lookup_key):
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key:
                    yield v
                else:
                    yield from item_generator(v, lookup_key)
        elif isinstance(json_input, list):
            for item in json_input:
                yield from item_generator(item, lookup_key)

    def tree_depth(json_text):
        json_input = json.loads(json_text)
        depths = list(item_generator(json_input, 'depth'))
        return max(depths) + 1 if len(depths) != 0 else 1

    train_acc = accuracy_score(y_train,xgb.predict(X_train))
    test_acc = accuracy_score(y_test,xgb.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

    booster = xgb.get_booster()

    tree_df = booster.trees_to_dataframe()
    depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
    num_trees = len(depths)
    depth = np.average(depths)
    num_nodes = len(tree_df)

    with open('./benchmark_xgb/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

In [9]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    clf = GradientBoostingClassifier(n_estimators=1000)
    # clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GBDT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

KeyboardInterrupt: 

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
for i in range(30):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    # y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)